In [1]:
import sys
sys.path.insert(0, '../')

import numpy as np
from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget
import time
import threading
import random

# Setup Mount Controller

In [2]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=2, address_search_depth=0)
    #KMC.swap_alice_bob()


ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM8', 'COM9']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount) (Address: 0)
Device succesfully connected (Rotation Mount) (Address: 0)


In [3]:
# Init the Time Tagger and the Kinetic Mount Controller
TTC = TimeTaggerController()

# NOTE Look at the trace, sucessively block each port and assign the channels numbers in the next cell
# NOTE afterwards stop and close the graphic
#TTC.displayCountTraces()

In [4]:
# connect KMC and TTC
TTC.setKineticMountController(KMC)
# set alice channels
TTC.set_alice_reflection_channel(2)
TTC.set_alice_transmission_channel(4)
# set bob channels
TTC.set_bob_reflection_channel(3)
TTC.set_bob_transmission_channel(1)


# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(9)
KMC.alice.home()



In [5]:
_ = TTC.performDelayAdjustment(integration_time=5)#, manual_delays=[0, 947, 78, -502])

Delays Before Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :   921 ps 	/   276.3 mm
Bob_T   :    72 ps 	/    21.6 mm
Bob_R   :  -508 ps 	/  -152.4 mm

Delays After Correction
Alice_T :     0 ps 	/     0.0 mm
Alice_R :    21 ps 	/     6.3 mm
Bob_T   :     4 ps 	/     1.2 mm
Bob_R   :    -7 ps 	/    -2.1 mm


In [6]:
KMC.rotate_simulataneously(0, 0)

In [7]:
n_values = 300
trace_width = 2
TTC.displayCountTraces(n_values=n_values, trace_width=trace_width, plot_title='<b>Raw Counts<b>')
TTC.displayCoincidenceTraces(coincidence_window_SI=0.5e-9, n_values=n_values, trace_width=trace_width, plot_title='<b>Coincidences<b>')

Output()

Output()

In [8]:
KMC.rotate_simulataneously(0, 0, wait_for_completion=False)

In [9]:

KMC.rotate_simulataneously(0, 45, wait_for_completion=False)

In [10]:
# TODO
# coincidence channel handling and destruction
#TTC.coincidences_vchannels.getChannels()

In [11]:
# TODO This kills the coincidence channels in the visualisation. Needs to be worked on
idealAngles = [0, -22.5, 11.25, -11.25] # for triplett |HH> - |VV> initial state
johannes_ideal_angles = [0, 45/2, -22.5/2, 22.5/2]
johannes_classical_angles = [0, 45/2, -90/2, -45/2]
TTC.measureS(CHSH_angles=idealAngles, integration_time_per_basis_setting_SI=1, TTSimulator=None, coincidence_window_SI=0.5e-9, debug=True)



corr[a,b] = 0.50753
	N[|T,T>]=   824	(0.365)
	N[|T,R>]=   485	(0.215)
	N[|R,T>]=    71	(0.031)
	N[|R,R>]=   878	(0.389)

corr[a,B] = 0.54452
	N[|T,T>]=   869	(0.389)
	N[|T,R>]=   389	(0.174)
	N[|R,T>]=   120	(0.054)
	N[|R,R>]=   857	(0.383)

corr[A,b] = 0.38002
	N[|T,T>]=   624	(0.230)
	N[|T,R>]=   453	(0.167)
	N[|R,T>]=   388	(0.143)
	N[|R,R>]=  1248	(0.460)

corr[A,B] = -0.72674
	N[|T,T>]=   237	(0.086)
	N[|T,R>]=  1512	(0.549)
	N[|R,T>]=   864	(0.314)
	N[|R,R>]=   139	(0.051)

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = 2.1588141039822593


In [12]:
TTC.measure_S_with_two_ports(CHSH_angles=idealAngles, debug=True)

SPCM Pairs: ['TT', 'TR', 'RT', 'RR']

corr[a,b] = [ 0.628 -0.612 -0.652  0.626]
	N[a , b ]=[809 491  81 881]
	N[a , b ]=[ 164 2135  467  187]
	N[a , b ]=[ 135 1450 1020  513]
	N[a , b ]=[ 499  373  232 2160]

corr[a,B] = [ 0.736 -0.704 -0.7    0.683]
	N[a , B ]=[840 380 101 895]
	N[a , B ]=[ 124 2197  403  211]
	N[a , B ]=[  87 1616 1048  369]
	N[a , B ]=[ 550  282  155 2179]

corr[A,b] = [ 0.417 -0.457 -0.488  0.539]
	N[A , b ]=[ 606  467  390 1287]
	N[A , b ]=[ 168 1717  621  749]
	N[A , b ]=[ 311 1569  724  128]
	N[A , b ]=[ 557  758   73 1639]

corr[A,B] = [-0.488  0.479  0.581 -0.594]
	N[A , B ]=[ 241 1421  973  109]
	N[A , B ]=[ 529  690   64 1818]
	N[A , B ]=[ 686  455  324 1086]
	N[A , B ]=[ 177 1830  491  630]

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = TT, TR, RT, RR : [2.26891673 2.25168725 2.42110618 2.44138378]


In [13]:
SimulatorPhiPlus = TT_Simulator(two_particle_states['phi_plus'], initial_state_noise=0.00, debug=False)
SimulatorPhiMinus = TT_Simulator(two_particle_states['phi_minus'], initial_state_noise=0.00, debug=False)
TTC.measureS(CHSH_angles=idealAngles, integration_time_per_basis_setting_SI=1, TTSimulator=SimulatorPhiPlus, coincidence_window_SI=0.5e-9)


corr[a,b] = 0.6956
	N[|T,T>]=  2135	(0.427)
	N[|T,R>]=   361	(0.072)
	N[|R,T>]=   400	(0.080)
	N[|R,R>]=  2104	(0.421)

corr[a,B] = 0.7004
	N[|T,T>]=  2137	(0.427)
	N[|T,R>]=   383	(0.077)
	N[|R,T>]=   366	(0.073)
	N[|R,R>]=  2114	(0.423)

corr[A,b] = -0.6964
	N[|T,T>]=   376	(0.075)
	N[|T,R>]=  2114	(0.423)
	N[|R,T>]=  2127	(0.425)
	N[|R,R>]=   383	(0.077)

corr[A,B] = 0.7144
	N[|T,T>]=  2208	(0.442)
	N[|T,R>]=   345	(0.069)
	N[|R,T>]=   369	(0.074)
	N[|R,R>]=  2078	(0.416)

S = abs(corrs[0,0] + corrs[0,1] + corrs[1,0] - corrs[1,1]) = 0.014800000000000146


# OSC Server Setup


In [18]:
my_ip = '192.168.0.2'
my_port = 7401 
clemens_port = 7401
visual_port = 7403

if 'OSC' not in globals():
    OSC = OSCCommunicator(my_ip=my_ip, my_port=my_port)

Myself = OSCTarget(ip=my_ip, port=my_port, name='Myself')
Clemens = OSCTarget(ip='192.168.0.3', port=clemens_port, name='Clemens')
Visuals = OSCTarget(ip=my_ip, port=visual_port, name='Visuals')

# Start the server
OSC.start_server()

In [19]:
# Send measurements
measurement_results = [1, 2, 3, 4]
OSC.send_measurement(target=Clemens, measurement_results=measurement_results)


Measurement [1, 2, 3, 4] sent to Clemens


In [20]:
OSC.send_visuals(target=Visuals, measurement_results="hi")

Measurement hi sent to Visuals


In [21]:
# Global flag to control the loop
performance_running = False
resultTranslate = {0 : [1, 1],
                   1 : [1, -1],
                   2 : [-1, 1],
                   3 : [-1, -1]}
# a A b B
Q_angles = [0, -22.5, 11.5, -11.5]
C_angles = [0, 22.5, -45, -22.5]

                    # ab, aB, Ab, AB
angle_pair_dict = {'Q_all':[(0, 11.5), (0, -11.5), (-22.5, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'Q_diagonal': [(0, -11.5), (-22.5, -11.5)],
                    # ab, AB
                   'C_diagonal': [(0, -45), (22.5, -22.5)],
                    # AB
                   'C_single': [(-45, -22.5)],
                   }

def performance_cycle():
    """The function you want to execute repeatedly."""
    global performance_running
    
    time_of_last_visual = time.time()
    while performance_running:
        theta_a, theta_b = random.choice(angle_pair_dict[OSC.currently_selected_state])
        
        measurement_results = TTC.get_single_measurement(theta_a, theta_b)
        
        # check if setup is measuring Quantum or Classical angles for conversion to clemens format        
        if OSC.currently_selected_state[0] == 'Q':
            check_angles = Q_angles
        elif OSC.currently_selected_state[0] == 'C':
            check_angles = C_angles

        resultClemensFormat = [1 if theta_a == check_angles[0] else 2, 1 if theta_b == check_angles[2] else 2 ,*resultTranslate[measurement_results]]

        OSC.send_measurement(Clemens, resultClemensFormat)

        #if time.time() - time_of_last_visual > 7:
        #    OSC.send_visuals(Visuals, resultClemensFormat)
        #    time_of_last_visual = time.time()


def start_performance():
    """Starts the task in a separate thread."""
    global performance_running
    performance_running = True  # Ensure the loop is enabled
    task_thread = threading.Thread(target=performance_cycle, daemon=True)
    task_thread.start()

def stop_performance():
    """Signals the task to stop gracefully."""
    global performance_running
    performance_running = False
    print('performance stopped...')


In [25]:
Myself.client.send_message("/bruQner/set/state", 'C_single')

In [26]:
stop_performance()

performance stopped...


In [22]:
start_performance()

Q_all
Measurement [1, 2, -1, -1] sent to Clemens
Q_all
Measurement [2, 1, -1, -1] sent to Clemens
Q_all
Measurement [1, 2, -1, -1] sent to Clemens
Q_all
Measurement [2, 2, -1, -1] sent to Clemens
Q_all
Measurement [2, 2, 1, 1] sent to Clemens
Q_all
Measurement [1, 1, 1, 1] sent to Clemens
Q_all
Measurement [1, 1, 1, 1] sent to Clemens
Q_diagonal
Measurement [2, 2, 1, -1] sent to Clemens
Q_diagonal
Measurement [1, 2, -1, -1] sent to Clemens
Q_diagonal
Measurement [1, 2, 1, 1] sent to Clemens
Q_diagonal
Measurement [2, 2, 1, -1] sent to Clemens
Q_diagonal
Measurement [1, 2, -1, 1] sent to Clemens
Q_diagonal
Measurement [1, 2, 1, 1] sent to Clemens
Q_diagonal
Measurement [1, 2, 1, -1] sent to Clemens
Q_diagonal
Measurement [1, 2, 1, 1] sent to Clemens
Q_diagonal
Measurement [2, 2, 1, -1] sent to Clemens
Q_diagonal
Measurement [2, 2, -1, 1] sent to Clemens
Q_diagonal
Measurement [1, 2, -1, -1] sent to Clemens
Q_diagonal
Measurement [1, 2, 1, 1] sent to Clemens
Q_diagonal
Measurement [1, 2,

Exception in thread Thread-84 (performance_cycle):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\ghana\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ghana\AppData\Local\Temp\ipykernel_9904\2309637255.py", line 27, in performance_cycle
KeyError: 'C_opas'


C_opas
Measurement [2, 2, -1, -1] sent to Clemens
